# Using _egobox_ optimizer _Egor_

## Installation

In [15]:
%pip install egobox

Note: you may need to restart the kernel to use updated packages.


We import _egobox_ as _egx_ for short.

In [1]:
import numpy as np
import egobox as egx

You may setup the logging level to get optimization progress during the execution

In [2]:
# To display optimization information (none by default)
import logging
logging.basicConfig(level=logging.INFO)

## Example 1 : Continuous optimization

### Test functions

In [3]:
xspecs_xsinx = egx.to_specs([[0., 25.]])
n_cstr_xsinx = 0

def xsinx(x: np.ndarray) -> np.ndarray:
    x = np.atleast_2d(x)
    y = (x - 3.5) * np.sin((x - 3.5) / (np.pi))
    return y

In [4]:
xspecs_g24 = egx.to_specs([[0., 3.], [0., 4.]])
n_cstr_g24 = 2

# Objective
def G24(point):
    """
    Function g24
    1 global optimum y_opt = -5.5080 at x_opt =(2.3295, 3.1785)
    """
    p = np.atleast_2d(point)
    return - p[:, 0] - p[:, 1]

# Constraints < 0
def G24_c1(point):
    p = np.atleast_2d(point)
    return (- 2.0 * p[:, 0] ** 4.0
            + 8.0 * p[:, 0] ** 3.0 
            - 8.0 * p[:, 0] ** 2.0 
            + p[:, 1] - 2.0)

def G24_c2(point):
    p = np.atleast_2d(point)
    return (-4.0 * p[:, 0] ** 4.0
            + 32.0 * p[:, 0] ** 3.0
            - 88.0 * p[:, 0] ** 2.0
            + 96.0 * p[:, 0]
            + p[:, 1] - 36.0)

# Grouped evaluation
def g24(point):
    p = np.atleast_2d(point)
    return np.array([G24(p), G24_c1(p), G24_c2(p)]).T


### Continuous optimization

In [5]:
egor = egx.Egor(xspecs_xsinx, n_cstr=n_cstr_xsinx)   # see help(egor) for options

In [14]:
egor = egx.Egor(xspecs_g24, 
                n_doe=10, 
                n_cstr=n_cstr_g24, 
                cstr_tol=[1e-3, 1e-3],
                infill_strategy=egx.InfillStrategy.WB2,
                target=-5.5,
                trego=False,
                # outdir="./out",
                # hot_start=True
            )   

# Specify regression and/or correlation models used to build the surrogates of objective and constraints
#egor = egx.Egor(g24, xlimits_g24, n_cstr=n_cstr_g24, n_doe=10,
#                      regr_spec=egx.RegressionSpec.LINEAR,
#                      corr_spec=egx.CorrelationSpec.MATERN32 | egx.CorrelationSpec.MATERN52)  

In [15]:
res = egor.minimize(g24, max_iters=30)
print(f"Optimization f={res.y_opt} at {res.x_opt}")
print("Optimization history: ")
print(f"Inputs = {res.x_doe}")
print(f"Outputs = {res.y_doe}")

INFO:egobox_ego.egor:EgorConfig { max_iters: 30, n_start: 20, n_optmod: 1, q_points: 1, n_doe: 10, n_cstr: 2, cstr_tol: Some([0.001, 0.001], shape=[2], strides=[1], layout=CFcf (0xf), const ndim=1), doe: None, q_ei: KrigingBeliever, infill_criterion: WB2, infill_optimizer: Cobyla, regression_spec: RegressionSpec(CONSTANT), correlation_spec: CorrelationSpec(SQUAREDEXPONENTIAL), kpls_dim: None, n_clusters: 1, target: -5.5, outdir: None, warm_start: false, hot_start: Disabled, xtypes: [Cont(0.0, 3.0), Cont(0.0, 4.0)], seed: None, trego: TregoConfig { activated: false, n_local_steps: 4, d: (1e-6, 1.0), beta: 0.9, gamma: 1.1111111111111112, sigma0: 0.1 } }
INFO:egobox_ego.solver.egor_solver:Compute initial LHS with 10 points
INFO:egobox_ego.solver.egor_impl:Train surrogates with 10 points...
INFO:egobox_ego.solver.egor_impl:Constraint[1] Clustering and training...
INFO:egobox_ego.solver.egor_impl:Constraint[2] Clustering and training...
INFO:egobox_ego.solver.egor_impl:Objective Clustering 

INFO:egobox_ego.solver.egor_impl:Infill criterion WB2 max found = inf
INFO:egobox_ego.solver.egor_impl:+1 point(s), total: 11 points
INFO:egobox_ego.solver.egor_solver:********* End iteration 1/30 in 0.050s: Best fun(x)=[-4.503768675148718, -2.772294135479884, -0.3577741240315433] at x=[2.47747031435393, 2.0262983607947884]
INFO:egobox_ego.solver.egor_impl:Train surrogates with 11 points...
INFO:egobox_ego.solver.egor_impl:Objective model hyperparameters optim init >>> [Optimized { init: [4.407757464854422e-6, 6.6329730464699226e-6], bounds: [(1e-8, 100.0)] }]
INFO:egobox_ego.solver.egor_impl:Infill criterion WB2 scaling is updated to 8.804656340337294
INFO:egobox_ego.solver.egor_impl:Constraints scalings is updated to [17.56984718519449, 35.521700892457886]
INFO:egobox_ego.solver.egor_impl:Optimize infill criterion...
INFO:egobox_ego.solver.egor_impl:Infill criterion WB2 max found = 0.7350888624584337
INFO:egobox_ego.solver.egor_impl:+1 point(s), total: 12 points
INFO:egobox_ego.solve

Optimization f=[-5.50730271e+00 -3.59983936e-03  4.55710257e-04] at [2.32983528 3.17746743]
Optimization history: 
Inputs = [[0.49612504 1.80492358]
 [0.07205367 0.55201681]
 [0.61897252 3.07984889]
 [1.76612527 1.52477595]
 [0.9965523  3.64490338]
 [2.06635074 3.20826396]
 [2.47747031 2.02629836]
 [1.22779255 0.23153528]
 [2.97564116 2.65606218]
 [2.2330071  1.19464179]
 [2.34402424 3.29108892]
 [2.32983528 3.17746743]]
Outputs = [[-2.30104862e+00 -1.30843675e+00 -4.56201736e+00]
 [-6.24070486e-01 -1.48657828e+00 -2.89758402e+01]
 [-3.69882141e+00 -3.81578930e-01 -2.12466342e-01]
 [-3.29090123e+00 -8.16448062e-01 -2.04961216e+00]
 [-4.64145568e+00 -3.55049078e-01  3.64471253e+00]
 [-5.27461470e+00  1.17066901e+00 -7.56594198e-01]
 [-4.50376868e+00 -2.77229414e+00 -3.57774124e-01]
 [-1.45932783e+00 -3.56629197e+00 -4.20345439e-01]
 [-5.63170334e+00 -1.62005905e+01  2.64679838e+00]
 [-3.42764888e+00 -1.34679591e+00 -2.38281038e+00]
 [-5.63511315e+00 -9.47670851e-03  1.81880897e-01]
 [-5

### Egor as a service: ask-and-tell interface

When the user needs to be in control of the optimization loop, `Egor` can be used as a service. 

For instance with the `xsinx` objective function, we can do:

In [16]:
xlimits = egx.to_specs([[0.0, 25.0]])
egor = egx.Egor(xlimits, seed=42) 

# initial doe
x_doe = egx.lhs(xlimits, 3, seed=42)
y_doe = xsinx(x_doe)
for _ in range(10): # run for 10 iterations
    x = egor.suggest(x_doe, y_doe)  # ask for best location
    x_doe = np.concatenate((x_doe, x))
    y_doe = np.concatenate((y_doe, xsinx(x))) 
res = egor.get_result(x_doe, y_doe)
print(f"Optimization f={res.y_opt} at {res.x_opt}")

INFO:egobox_ego.solver.egor_impl:Train surrogates with 3 points...
INFO:egobox_ego.solver.egor_impl:Objective Clustering and training...
INFO:egobox_ego.solver.egor_impl:... Objective trained (1 / Mixture[Smooth(1)])
INFO:egobox_ego.solver.egor_impl:Infill criterion WB2 scaling is updated to 13.125691505249769
INFO:egobox_ego.solver.egor_impl:Optimize infill criterion...
INFO:egobox_ego.solver.egor_impl:Train surrogates with 4 points...
INFO:egobox_ego.solver.egor_impl:Objective Clustering and training...
INFO:egobox_ego.solver.egor_impl:... Objective trained (1 / Mixture[Smooth(1)])
INFO:egobox_ego.solver.egor_impl:Infill criterion WB2 scaling is updated to 24.009123072353827
INFO:egobox_ego.solver.egor_impl:Optimize infill criterion...
INFO:egobox_ego.solver.egor_impl:Train surrogates with 5 points...
INFO:egobox_ego.solver.egor_impl:Objective Clustering and training...
INFO:egobox_ego.solver.egor_impl:... Objective trained (1 / Mixture[Smooth(1)])
INFO:egobox_ego.solver.egor_impl:In

Optimization f=[-15.12510279] at [18.9344756]


## Example 2 : Mixed-integer optimization

### Test function

In [17]:
xspecs_mixint_xsinx = [egx.XSpec(egx.XType.INT, [0, 25])]
n_cstr_mixint_xsinx = 0

def mixint_xsinx(x: np.ndarray) -> np.ndarray:
    x = np.atleast_2d(x)
    if (np.abs(np.linalg.norm(np.floor(x))-np.linalg.norm(x))< 1e-8):
        y = (x - 3.5) * np.sin((x - 3.5) / (np.pi))
    else:
        raise ValueError(f"Bad input: mixint_xsinx accepts integer only, got {x}")
    return y

### Mixed-integer optimization with _Egor_

In [18]:
egor = egx.Egor(xspecs_mixint_xsinx, 
                     n_doe=3, 
                     infill_strategy=egx.InfillStrategy.EI,
                     target=-15.12,
                    )  # see help(egor) for options
res = egor.minimize(mixint_xsinx, max_iters=30)
print(f"Optimization f={res.y_opt} at {res.x_opt}")
print("Optimization history: ")
print(f"Inputs = {res.x_doe}")
print(f"Outputs = {res.y_doe}")

INFO:egobox_ego.egor:EgorConfig { max_iters: 30, n_start: 20, n_optmod: 1, q_points: 1, n_doe: 3, n_cstr: 0, cstr_tol: Some([], shape=[0], strides=[0], layout=CFcf (0xf), const ndim=1), doe: None, q_ei: KrigingBeliever, infill_criterion: EI, infill_optimizer: Cobyla, regression_spec: RegressionSpec(CONSTANT), correlation_spec: CorrelationSpec(SQUAREDEXPONENTIAL), kpls_dim: None, n_clusters: 1, target: -15.12, outdir: None, warm_start: false, hot_start: Disabled, xtypes: [Int(0, 25)], seed: None, trego: TregoConfig { activated: false, n_local_steps: 4, d: (1e-6, 1.0), beta: 0.9, gamma: 1.1111111111111112, sigma0: 0.1 } }
INFO:egobox_ego.solver.egor_solver:Compute initial LHS with 3 points
INFO:egobox_ego.solver.egor_impl:Train surrogates with 3 points...
INFO:egobox_ego.solver.egor_impl:Objective Clustering and training...
INFO:egobox_ego.solver.egor_impl:... Objective trained (1 / Mixture[Smooth(1)])
INFO:egobox_ego.solver.egor_impl:Infill criterion EI scaling is updated to 0.446585115

INFO:egobox_ego.solver.egor_impl:Infill criterion EI scaling is updated to 0.5619422973386059
INFO:egobox_ego.solver.egor_impl:Optimize infill criterion...
INFO:egobox_ego.solver.egor_impl:Infill criterion EI max found = 1
INFO:egobox_ego.solver.egor_impl:+1 point(s), total: 5 points
INFO:egobox_ego.solver.egor_solver:********* End iteration 2/30 in 0.017s: Best fun(x)=[-12.352951424113456] at x=[17.055840869536123]
INFO:egobox_ego.solver.egor_impl:Train surrogates with 5 points...
INFO:egobox_ego.solver.egor_impl:Objective model hyperparameters optim init >>> [Optimized { init: [19.826278888584234], bounds: [(1e-8, 100.0)] }]
INFO:egobox_ego.solver.egor_impl:Infill criterion EI scaling is updated to 1.312744230503256
INFO:egobox_ego.solver.egor_impl:Optimize infill criterion...
INFO:egobox_ego.solver.egor_impl:Infill criterion EI max found = 1
INFO:egobox_ego.solver.egor_impl:+1 point(s), total: 6 points
INFO:egobox_ego.solver.egor_solver:********* End iteration 3/30 in 0.015s: Best f

Optimization f=[-15.12161154] at [19.]
Optimization history: 
Inputs = [[16.]
 [ 6.]
 [23.]
 [15.]
 [17.]
 [18.]
 [19.]]
Outputs = [[ -9.28531859]
 [  1.78601478]
 [ -1.48334497]
 [ -5.7038496 ]
 [-12.35295142]
 [-14.43198471]
 [-15.12161154]]


## Example 3 : More mixed-integer optimization

In the following example we see we can have other special integer type cases, where a component of x can take one value out of a list of ordered values (ORD type) or being like an enum value (ENUM type). Those types differ by the processing related to the continuous relaxation made behind the scene:
* For INT type, resulting float is rounded to the closest int value,
* For ORD type, resulting float is cast to closest value among the given valid ones,
* For ENUM type, one hot encoding is performed to give the resulting value.  

### Test function

In [19]:
# Objective function which takes [FLOAT, ENUM1, ENUM2, ORD] as input
# Note that ENUM values are passed as indice value eg either 0, 1 or 2 for a 3-sized enum  
def mixobj(X):
    # float
    x1 = X[:, 0]
    #  ENUM 1
    c1 = X[:, 1]
    x2 = c1 == 0
    x3 = c1 == 1
    x4 = c1 == 2
    #  ENUM 2
    c2 = X[:, 2]
    x5 = c2 == 0
    x6 = c2 == 1
    # int
    i = X[:, 3]

    y = (x2 + 2 * x3 + 3 * x4) * x5 * x1 + (x2 + 2 * x3 + 3 * x4) * x6 * 0.95 * x1 + i
    return y.reshape(-1, 1)

### Mixed-integer optimization with _Egor_

In [20]:
xtypes = [
    egx.XSpec(egx.XType.FLOAT, [-5.0, 5.0]),
    egx.XSpec(egx.XType.ENUM, tags=["blue", "red", "green"]),
    egx.XSpec(egx.XType.ENUM, xlimits=[2]),
    egx.XSpec(egx.XType.ORD, [0, 2, 3]),
]
egor = egx.Egor(xtypes, seed=42)
res = egor.minimize(mixobj, max_iters=10)
print(f"Optimization f={res.y_opt} at {res.x_opt}")
print("Optimization history: ")
print(f"Inputs = {res.x_doe}")
print(f"Outputs = {res.y_doe}")

INFO:egobox_ego.egor:EgorConfig { max_iters: 10, n_start: 20, n_optmod: 1, q_points: 1, n_doe: 0, n_cstr: 0, cstr_tol: Some([], shape=[0], strides=[0], layout=CFcf (0xf), const ndim=1), doe: None, q_ei: KrigingBeliever, infill_criterion: WB2, infill_optimizer: Cobyla, regression_spec: RegressionSpec(CONSTANT), correlation_spec: CorrelationSpec(SQUAREDEXPONENTIAL), kpls_dim: None, n_clusters: 1, target: -inf, outdir: None, warm_start: false, hot_start: Disabled, xtypes: [Cont(-5.0, 5.0), Enum(3), Enum(2), Ord([0.0, 2.0, 3.0])], seed: Some(42), trego: TregoConfig { activated: false, n_local_steps: 4, d: (1e-6, 1.0), beta: 0.9, gamma: 1.1111111111111112, sigma0: 0.1 } }
INFO:egobox_ego.solver.egor_solver:Compute initial LHS with 8 points
INFO:egobox_ego.solver.egor_impl:Train surrogates with 8 points...
INFO:egobox_ego.solver.egor_impl:Objective Clustering and training...
INFO:egobox_ego.solver.egor_impl:... Objective trained (1 / Mixture[Smooth(1)])
INFO:egobox_ego.solver.egor_impl:Infil

Optimization f=[-14.25] at [-5.  2.  1.  0.]
Optimization history: 
Inputs = [[ 0.69939824  0.          0.          0.        ]
 [ 4.84411847  1.          0.          0.        ]
 [-4.75038813  1.          0.          2.        ]
 [-1.81967258  2.          1.          2.        ]
 [ 2.46052467  0.          0.          2.        ]
 [-2.82859054  0.          0.          2.        ]
 [ 2.5012666   2.          1.          0.        ]
 [-0.6935668   2.          1.          3.        ]
 [-4.77212695  2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]]
Outp

Note that `x_opt` result contains indices for corresponding optional tags list hence the second component should be read as 0="red", 1="green", 2="blue", while the third component was unamed 0 correspond to first enum value and 1 to the second one.

## Usage

In [21]:
help(egor)

Help on Egor in module builtins object:

class Egor(object)
 |  Egor(xspecs, n_cstr=0, cstr_tol=None, n_start=20, n_doe=0, doe=None, regr_spec=Ellipsis, corr_spec=Ellipsis, infill_strategy=Ellipsis, q_points=1, par_infill_strategy=Ellipsis, infill_optimizer=Ellipsis, kpls_dim=None, trego=False, n_clusters=1, n_optmod=1, target=Ellipsis, outdir=None, warm_start=False, hot_start=None, seed=None)
 |  
 |  Optimizer constructor
 |  
 |  fun: array[n, nx]) -> array[n, ny]
 |       the function to be minimized
 |       fun(x) = [obj(x), cstr_1(x), ... cstr_k(x)] where
 |          obj is the objective function [n, nx] -> [n, 1]
 |          cstr_i is the ith constraint function [n, nx] -> [n, 1]
 |          an k the number of constraints (n_cstr)
 |          hence ny = 1 (obj) + k (cstrs)
 |       cstr functions are expected be negative (<=0) at the optimum.
 |  
 |   n_cstr (int):
 |       the number of constraint functions.
 |  
 |   cstr_tol (list(n_cstr,)):
 |       List of tolerances for 